In [ ]:
import pandas as pd
import re

In [ ]:
#specialcasefood={'cilok':'aci','cimol':'aci','cilor':'aci','cireng':'aci','cilung':'aci','cakwe':'gorengan lain','basreng':'gorengan lain','sate kikil':'sate','soto':'soto/sop/makanan berkuah lainnya','kebab turki':'kebab'}
#array buat kasus khusus yg ribet diganti pake regex
specialcasefood={'soto':'soto/sop/makanan berkuah lainnya'}
specialcasedrink={}
stupidcase=['etc','random','']

In [ ]:
file_dir='kaki.xlsx'

In [ ]:
data=pd.read_excel(file_dir)

In [ ]:
data

In [ ]:
#converting into array per entry
foodarr=data['Kalau jajan di pedagang kecil, Kamu sukanya jajanan makanan apa? (Bisa isi lebih dari satu)'].str.lower().str.strip(' ')
#regexing all the stupid entry
foodarr=foodarr.replace(r"ci[a-z]*","aci",regex=True)
foodarr=foodarr.replace(r"(cakwe|basreng)","gorengan lain",regex=True)
foodarr=foodarr.replace(r"sate[a-z ]*","sate",regex=True)
#foodarr=foodarr.replace(r"(soto [a-z ]*|sop [a-z ]*)","soto/sop/makanan berkuah lainnya",regex=True)
foodarr=foodarr.replace(r"kebab[a-z ]*","kebab",regex=True)
foodarr=foodarr.replace(r"soto$","soto/sop/makanan berkuah lainnya",regex=True)

In [ ]:
#foodarr.unique()

In [ ]:
#menghitung total makanan dari kuesioner ke dictionary
fooddict={}
for i in foodarr:
    #biar engga duplikat
    temp=dict.fromkeys(i.split(', '))
    #split karena data terpisah oleh koma
    for j in temp:
        if j in stupidcase:
            continue
        if j not in specialcasefood:
            if j in fooddict:
                fooddict[j]=fooddict[j]+1
            else:
                fooddict[j]=1
        else:
            entry=specialcasefood[j]
            if entry in fooddict:
                fooddict[entry]=fooddict[entry]+1
            else:
                fooddict[entry]=1

In [ ]:
dict(sorted(fooddict.items(), key=lambda item: item[1]))

In [ ]:
food_cat=pd.CategoricalIndex(fooddict)

In [ ]:
numbered_foodarr=[]
for i in foodarr:
    #print('baris ',i)
    temparr=[]
    for j in i.split(', '):
        if j in stupidcase:
            continue
        temparr.append(food_cat.get_loc(j))
    numbered_foodarr.append(temparr)

MINUMAN

In [ ]:
drinkarr=data['Kalau jajan di pedagang kecil, Kamu sukanya minuman apa? (Bisa isi lebih dari satu)'].str.lower().str.strip(' ')
#drinkarr.unique()

In [ ]:
#replacing all the different entry
drinkarr=drinkarr.replace(r"wedang[a-z ]*","wedang",regex=True)
drinkarr=drinkarr.replace(r"(air[a-z ]*|mineral[a-z ]*|aqua[a-z ]*)","minuman air",regex=True)
drinkarr=drinkarr.replace(r"([a-z][a-z ]*latte[a-z ]*|capucino[a-z ]*)","kopi",regex=True)
drinkarr=drinkarr.replace(r"[a-z][a-z ]*jeruk","jus buah",regex=True)
drinkarr=drinkarr.replace(r"teh[a-z ]*","teh",regex=True)
drinkarr=drinkarr.replace(r"(pop ice|minuman saset)","soda/soft drink",regex=True)
drinkarr=drinkarr.replace(r"es boba","bubble tea",regex=True)
drinkarr=drinkarr.replace(r"susu[a-z ]*","susu",regex=True)
drinkarr=drinkarr.replace(r"(es degan[a-z ]*|es kelapa[a-z ]*)","air kelapa",regex=True)
drinkarr=drinkarr.replace(r"random","",regex=True)

In [ ]:
drinkarr

In [ ]:
drinkdict={}
for i in drinkarr:
    #biar engga duplikat
    temp=dict.fromkeys(i.split(', '))
    for j in temp:
        if j in stupidcase:
            continue
        if j not in specialcasedrink:
            if j in drinkdict:
                drinkdict[j]=drinkdict[j]+1
            else:
                drinkdict[j]=1
        else:
            entry=specialcasedrink[j]
            if entry in drinkdict:
                drinkdict[entry]=drinkdict[entry]+1
            else:
                drinkdict[entry]=1

In [ ]:
dict(sorted(drinkdict.items(), key=lambda item: item[1]))

In [ ]:
drink_cat=pd.CategoricalIndex(drinkdict)

In [ ]:
numbered_drinkarr=[]
for i in drinkarr:
    #print('baris ',i)
    temparr=[]
    for j in i.split(', '):
        if j in stupidcase:
            continue
        temparr.append(drink_cat.get_loc(j))
    numbered_drinkarr.append(temparr)

In [ ]:
output=pd.DataFrame([range(1,len(numbered_drinkarr)+1),numbered_foodarr,numbered_drinkarr]).T.rename({0:'No',1:'makanan',2:'minuman'},axis=1)

In [ ]:
output.to_csv('output.csv',index=False)